In [1]:
from pyscipopt import Model,quicksum, multidict
import numpy as np
import pandas as pd
import openpyxl
import datetime as dt
import os

### 获取小类销售数据

In [2]:
ospwd = os.getcwd()

In [3]:
# data = pd.read_excel(ospwd+'\\lining.xlsx')
data = pd.read_excel(ospwd+'\LiNingSaleData2nd.xlsx')

In [4]:
# data

In [5]:
data.rename(columns={'小类':'class_2','尺码':'size','销量':'qty','占比':'ratio','子公司编码':'org_id','大类':'class_0','子公司名':'org_name','性别':'sex'},inplace = True)

In [6]:
data

,class_0,class_2,org_id,org_name,size,qty,sex,ratio
0,服装,1201,A21089A,大连悦奥体育用品销售有限公司,XL,2,男,0.666667
1,服装,1201,A21089A,大连悦奥体育用品销售有限公司,L,3,男,1.000000
2,服装,1201,A32028A,南京一动体育用品销售有限公司,XXL,1,男,0.031250
3,服装,1201,A32028A,南京一动体育用品销售有限公司,XL,31,男,0.968750
4,服装,1201,A32028A,南京一动体育用品销售有限公司,M,19,男,0.593750
5,服装,1201,A32028A,南京一动体育用品销售有限公司,3XL,5,男,0.156250
6,服装,1201,A32028A,南京一动体育用品销售有限公司,L,32,男,1.000000
7,服装,1201,A33116A,温州一动体育用品有限公司,M,9,男,0.264706
8,服装,1201,A33116A,温州一动体育用品有限公司,XL,34,男,1.000000
9,服装,1201,A33116A,温州一动体育用品有限公司,XXL,3,男,0.088235


In [7]:
class_list = data['class_2'].drop_duplicates().reset_index().drop('index',axis =1)
class_arry = class_list['class_2'].values

In [8]:
class_arry

array([1201, '2合1运动短裤（外-运动短裤/内-紧身长裤）', '背心', '比赛裤', '比赛上衣', '长棉服',
       '长袖Polo', '长袖T恤', '长袖紧身衣', '长袖套头衫', '长袖文化衫', '长羽绒服', '短棉服',
       '短袖Polo', '短袖T恤', '短袖文化衫', '短羽绒服', '夹克', '紧身运动长裤', '九分卫裤',
       '开衫连帽卫衣', '开衫连帽摇粒绒卫衣', '开衫无帽编织衫', '开衫无帽卫衣', '开衫无帽卫衣套装', '连衣裙',
       '棉夹克', '软壳长裤', '三合一(内-短棉服/外-运动风衣)', '三合一(内-紧身运动长裤/外-运动短裤)',
       '三合一(内-开衫无帽摇粒绒卫衣/外-运动风衣)', '速干长裤', '套头连帽卫衣', '套头无帽编织衫', '套头无帽卫衣',
       '卫裤', '休闲长裤', '休闲外套', '羽绒马甲', '运动长裤', '运动短裤', '运动风衣', '运动文胸',
       '针织长袖衫', '针织阔腿裤', '针织外套', '针织运动裤', '针织运动上衣', '中棉服', '中羽绒服', '潮流拖鞋',
       '潮流休闲鞋', '户外休闲鞋', '缓震训练鞋', '减震跑鞋', '经典休闲鞋', '篮球场地鞋', '篮球休闲鞋',
       '篮球专业比赛鞋', '灵动跑鞋', '灵活训练鞋', '轻质跑鞋', '稳定跑鞋', '羽毛球训练鞋', '羽毛球专业鞋'],
      dtype=object)

In [9]:
# clothes_size
class_size = pd.read_excel(ospwd +'\\ClassSize2nd.xlsx')

In [10]:
class_size

,brand,gender,tiny_class,size_id
0,LI-NING,童,长羽绒服,6XL
1,LI-NING,女,运动风衣,7XL
2,LI-NING,男,套头连帽卫衣,6XL
3,LI-NING,男,文化衫,4XS
4,LI-NING,男,经典休闲鞋,150
5,LI-NING,男,篮球外场鞋,6.5
6,LI-NING,男,长袖Polo,XXS
7,LI-NING,中,其他配饰,3XL
8,LI-NING,男,野外跑鞋,6.5
9,LI-NING,女,户外休闲鞋,8


In [11]:
class_size.rename(columns={'gender':'sex','tiny_class':'class_2','size_id':'size'},inplace = True)

In [12]:
# class_size = class_size[class_size['class_0'].isin(['服装','鞋'])] #删除其他大类

In [13]:
# class_size.loc[class_size['class_2']==1201]

In [14]:
# clo_data_group = ['class_2','org_id','size','qty','ratio','org_name']

In [15]:
# cloth_size_data = class_size.loc[(class_size['class_0'] =='服装'),['class_2','size']].reset_index().drop('index',axis=1)

size_data = class_size.reset_index().drop('index',axis=1)

In [16]:
size_data = size_data.drop('brand',axis=1)

In [17]:
size_data

,sex,class_2,size
0,童,长羽绒服,6XL
1,女,运动风衣,7XL
2,男,套头连帽卫衣,6XL
3,男,文化衫,4XS
4,男,经典休闲鞋,150
5,男,篮球外场鞋,6.5
6,男,长袖Polo,XXS
7,中,其他配饰,3XL
8,男,野外跑鞋,6.5
9,女,户外休闲鞋,8


In [18]:
# sale_data = data[('class_0','class_2','org_name','org_id')].drop_duplicates()
sale_data = data.loc[:,['class_0','class_2','org_name','org_id']].drop_duplicates()

In [19]:
sale_data

,class_0,class_2,org_name,org_id
0,服装,1201,大连悦奥体育用品销售有限公司,A21089A
2,服装,1201,南京一动体育用品销售有限公司,A32028A
7,服装,1201,温州一动体育用品有限公司,A33116A
13,服装,1201,厦门悦奥商贸有限公司,A35020A
19,服装,1201,长沙一动体育用品销售有限公司,A43001A
25,服装,1201,深圳悦奥商贸有限公司,A44190A
29,服装,1201,南宁一动体育用品销售有限公司,A45001A
33,服装,1201,重庆悦奥体育用品销售有限公司,A51210A
39,服装,1201,新疆一动体育用品销售有限公司,A65026A
43,服装,2合1运动短裤（外-运动短裤/内-紧身长裤）,天津一动体育用品销售有限公司,A12001A


In [20]:
sale_data['value']=1

In [21]:
size_data ['value']=1

In [22]:
# size_data

In [23]:
# Crt_prod=pd.merge(a1,cloth_size_data,how='right',on=['value','class_2'])
Crt_prod=pd.merge(size_data,sale_data,how='right',on=['value','class_2'])

In [24]:
Crt_prod

,sex,class_2,size,value,class_0,org_name,org_id
0,童,长羽绒服,6XL,1,服装,北京李宁体育用品销售有限公司,A11001A
1,中,长羽绒服,XL,1,服装,北京李宁体育用品销售有限公司,A11001A
2,女,长羽绒服,XS,1,服装,北京李宁体育用品销售有限公司,A11001A
3,中,长羽绒服,M,1,服装,北京李宁体育用品销售有限公司,A11001A
4,女,长羽绒服,9XL,1,服装,北京李宁体育用品销售有限公司,A11001A
5,女,长羽绒服,5XL,1,服装,北京李宁体育用品销售有限公司,A11001A
6,男,长羽绒服,3XL,1,服装,北京李宁体育用品销售有限公司,A11001A
7,男,长羽绒服,6XL,1,服装,北京李宁体育用品销售有限公司,A11001A
8,中,长羽绒服,5XL,1,服装,北京李宁体育用品销售有限公司,A11001A
9,女,长羽绒服,10XL,1,服装,北京李宁体育用品销售有限公司,A11001A


In [25]:
Crt_prod.drop('value',axis=1,inplace = True)

### 获得每个小类的全尺码表

In [26]:
Crt_prod.dropna(inplace =True)

In [27]:
# Crt_prod

In [28]:
# Crt_prod.to_excel('C:\\Users\\liuju\\Desktop\\readcode\\ljtext\\putdata.xlsx')

In [29]:
class_num = Crt_prod['class_2'].drop_duplicates()

In [30]:
# class_num

In [31]:
class_num.count()

65

In [32]:
# sale_class_num = clo_data['class_2'].drop_duplicates()

In [33]:
# Crt_prod = Crt_prod.loc[Crt_prod['class_2']=='背心']

In [34]:
# data = data.loc[data['class_2']=='背心']

In [35]:
data_result = pd.merge(Crt_prod,data,how = 'left',on = ['class_0','class_2','org_id','org_name','size','sex'])

In [36]:
data_result

,sex,class_2,size,class_0,org_name,org_id,qty,ratio
0,童,长羽绒服,6XL,服装,北京李宁体育用品销售有限公司,A11001A,NaN,NaN
1,中,长羽绒服,XL,服装,北京李宁体育用品销售有限公司,A11001A,89.0,1.000000
2,女,长羽绒服,XS,服装,北京李宁体育用品销售有限公司,A11001A,1.0,0.003546
3,中,长羽绒服,M,服装,北京李宁体育用品销售有限公司,A11001A,88.0,0.988764
4,女,长羽绒服,9XL,服装,北京李宁体育用品销售有限公司,A11001A,NaN,NaN
5,女,长羽绒服,5XL,服装,北京李宁体育用品销售有限公司,A11001A,NaN,NaN
6,男,长羽绒服,3XL,服装,北京李宁体育用品销售有限公司,A11001A,169.0,0.449468
7,男,长羽绒服,6XL,服装,北京李宁体育用品销售有限公司,A11001A,NaN,NaN
8,中,长羽绒服,5XL,服装,北京李宁体育用品销售有限公司,A11001A,NaN,NaN
9,女,长羽绒服,10XL,服装,北京李宁体育用品销售有限公司,A11001A,NaN,NaN


In [37]:
# data_result.set_index(['org_name','org_id'])
# data_result

In [38]:
# 获取填充以后的数据
data_result.fillna(0,inplace= True)

In [39]:
# data_result

In [40]:
data_result = data_result[['class_0','class_2','org_id','org_name','sex','size','qty','ratio']]

In [41]:
# data_result

In [42]:
localdata = ospwd +'\\Dealed_AllData.xlsx'
data_result.to_excel(localdata, index = False)